In [1]:
import os
import h5py
import numpy as np

input_datafile = h5py.File('../input/uploads/uploads/tryp')

train_norm_images = np.array(input_datafile['train_norm'])
train_trypo_images = np.array(input_datafile['train_trypo'])
valid_norm_images = np.array(input_datafile['valid_norm'])
valid_trypo_images = np.array(input_datafile['valid_trypo'])

print(train_norm_images[0])
print(train_trypo_images[0])
print(valid_norm_images[0])
print(valid_trypo_images[0])

[[[212 168   1]
  [212 168   1]
  [212 168   1]
  ..., 
  [216 163   0]
  [222 166   0]
  [218 162   0]]

 [[212 168   1]
  [212 168   1]
  [212 168   1]
  ..., 
  [212 161   2]
  [217 163   2]
  [211 156   1]]

 [[211 167   0]
  [211 167   0]
  [211 167   0]
  ..., 
  [209 160   9]
  [214 163   8]
  [207 155   2]]

 ..., 
 [[125  76   6]
  [125  76   6]
  [125  75   6]
  ..., 
  [190 136   0]
  [190 136   0]
  [190 137   0]]

 [[125  76   6]
  [125  76   6]
  [125  75   6]
  ..., 
  [190 136   0]
  [190 136   0]
  [190 136   0]]

 [[125  77   5]
  [126  77   5]
  [126  76   5]
  ..., 
  [190 134   0]
  [191 135   0]
  [191 135   0]]]
[[[ 39  55  76]
  [ 46  59  81]
  [ 47  59  78]
  ..., 
  [ 40  41  45]
  [ 37  37  43]
  [ 34  34  40]]

 [[ 37  50  73]
  [ 39  52  73]
  [ 45  57  75]
  ..., 
  [ 38  36  43]
  [ 41  39  47]
  [ 36  30  40]]

 [[ 42  54  79]
  [ 41  53  75]
  [ 42  53  70]
  ..., 
  [ 34  33  42]
  [ 36  32  43]
  [ 34  27  40]]

 ..., 
 [[ 58  63 101]
  [ 65  71 106]


In [2]:
import matplotlib.pyplot as plt
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
from scipy.misc import imresize

def get_display_image(image):
    plt.figure(figsize=(2, 2))
    plt.imshow(image[:, :, [2, 1, 0]])
    plt.show()

def res_to_VGG(image):
    for i in range(len(image)):
        image[i] = cv2.resize(image[i], (224, 224))
    return image
    
def process_images(image):
    return np.divide(image, 255)

def permutate_data(norm, trypo):
    print(norm.shape)
  #  norm = process_images(np.array(norm))
   # trypo = process_images(np.array(trypo))
    X = np.concatenate([norm, trypo])
    Y1 = np.zeros(len(norm))
    Y2 = np.ones(len(trypo))
    Y = np.concatenate([Y1, Y2])
    ind = [i for i in range(len(Y))]
    shuffle(ind)
    X_new  = X[ind, :,:,:]
    Y_new = np_utils.to_categorical(Y[ind]) 
    print(Y_new.shape)
    print(X_new.shape)
    return X_new, Y_new

X, Y = permutate_data(train_norm_images, train_trypo_images)
X_val, Y_val = permutate_data(valid_norm_images, valid_trypo_images)

print(X.shape)

Using TensorFlow backend.


(10068, 224, 224, 3)
(15884, 2)
(15884, 224, 224, 3)
(500, 224, 224, 3)
(1000, 2)
(1000, 224, 224, 3)
(15884, 224, 224, 3)


In [3]:
print(X[2])

[[[ 38  81 153]
  [ 37  78 142]
  [ 33  71 130]
  ..., 
  [  1   8  25]
  [  3  11  30]
  [  3  13  36]]

 [[ 51  97 171]
  [ 42  86 152]
  [ 39  81 142]
  ..., 
  [  0   7  22]
  [  2  11  27]
  [  5  16  36]]

 [[ 49  99 172]
  [ 48  95 163]
  [ 40  83 147]
  ..., 
  [  0   8  21]
  [  4  13  28]
  [  7  19  37]]

 ..., 
 [[  0   4  15]
  [  3   8  27]
  [  8  16  42]
  ..., 
  [ 19  48  98]
  [ 15  50 104]
  [ 13  47 105]]

 [[  2   4  15]
  [  4   7  28]
  [ 10  16  43]
  ..., 
  [ 22  53 102]
  [ 36  70 123]
  [ 10  44  99]]

 [[  2   4  16]
  [  4   8  30]
  [  8  17  43]
  ..., 
  [ 63  93 139]
  [ 30  62 112]
  [ 23  55 106]]]


In [4]:
import keras
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications.vgg16 import VGG16

model = VGG16(include_top = False, weights='imagenet')
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation = 'softmax')(x)
model = Model(model.input, x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [ ]:
for layer in model.layers[:-1]:
    layer.trainable = False
model.compile(optimizer = "adam", loss = "categorical_crossentropy",
             metrics = ["accuracy"])
model.evaluate(X_val, Y_val)


1000/1000 [==============================] - 22s    


[2.6868548049926759, 0.53200000000000003]

In [ ]:
model.fit(X, Y, epochs = 10, validation_data=[X_val, Y_val])

Train on 15884 samples, validate on 1000 samples
Epoch 1/10
10464/15884 [==================>...........] - ETA: 91s - loss: 0.7055 - acc: 0.8673

In [ ]:
model.save_weights("vgg_0.1_weights")